In [1]:
import json
from Utils.Helper import CreateFolder

text_file_location = "Philosophy Book.txt"
json_file_location = "Philosophy Book.json"

SHORTS_PATH = "Philosophy Shorts"
AUDIOS_PATH = "Sound Clips"

CreateFolder(SHORTS_PATH)
CreateFolder(AUDIOS_PATH)

# Prepsrocess Book

In [10]:
import re

def ParsePhilosophyBook(file_name):
    with open(file_name, 'r', encoding='utf-8') as f:
        data = f.read()

    # Split the text into chapters
    chapters = data.split('Chapter ')

    # Remove possible empty strings
    chapters = list(filter(None, chapters))

    parsed_chapters = []

    for chapter in chapters:
        chapter_dict = {}
        # Use regular expressions (regex) to find and extract required info
        sections = re.split(r'(Introduction|Life and Influence|Philosophical Ideas|Impact on Philosophy|Story: .*)', chapter)

        # Remove possible empty strings
        sections = list(filter(None, sections))

        # Extract chapter title and dates
        title_search = re.search(r'(\d+): (.*?) \((\d+)-(\d+) (.*?)\)', sections[0])
        chapter_dict['Title'] = title_search.group(2) + ' (' + title_search.group(3) + '-' + title_search.group(4) + f" {title_search.group(5)})"
        chapter_dict['Start'] = int(title_search.group(3))
        chapter_dict['End'] = int(title_search.group(4))
        chapter_dict['Time'] = title_search.group(5)

        chapter_dict['Sections'] = []

        for i in range(1,len(sections),2):
            section_dict = {}
            section_dict['Title'] = sections[i]
            section_dict['Text'] = sections[i + 1].strip()
            chapter_dict['Sections'].append(section_dict)

        parsed_chapters.append(chapter_dict)

    return parsed_chapters

def SortByTime(chapter):
    # For BC times, we invert the value to sort in chronological order
    start = -chapter['Start'] if chapter['Time'] == 'BC' else chapter['Start']
    return start

chapters = ParsePhilosophyBook(text_file_location)
chapters = sorted(chapters, key=SortByTime)

with open(json_file_location, 'w+') as file:
    json.dump(chapters, file, indent=4)

# Building Videos

In [2]:
with open(json_file_location, 'r') as file:
    chapters = json.load(file)

In [3]:
from IPython.display import clear_output
from Utils.BookToVideoFunctions import ChapterToVideo

n_chapters = len(chapters)
for idx, chapter in enumerate(chapters):
    if idx < 18:
        continue
    print(f"{idx+1}/{n_chapters}")
    _ = ChapterToVideo(chapter, idx, AUDIOS_PATH, SHORTS_PATH)
    clear_output()